# Overview

This notebook reads in all necessary datasources for the project and does some preliminary processing (clipping to the AOI, merging data layers).

# Prerequisites

## Necessary data files

| Data | Source | Notes |
| :- | -: | :-: |
| TIGER roads | https://uw-mad.maps.arcgis.com/home/item.html?id=f42ecc08a3634182b8678514af35fac3 | Primary, secondary, and local
| USFS roads | https://data.fs.usda.gov/geodata/edw/datasets.php?xmlKeyword=roads | Downloaded as geodatabase
| AOI - San Gabriel Natl Monument | https://uw-mad.maps.arcgis.com/home/item.html?id=8ee200037f6d4f3c9d362878beed7e0e | Exported as CAD files
| Visitor centers | https://data-usfs.hub.arcgis.com/datasets/usfs::recreation-opportunities-feature-layer/about/ & https://data-usfs.hub.arcgis.com/datasets/usfs::recreation-sites-feature-layer/about/| NA
| Recreation data | https://uw-mad.maps.arcgis.com/home/item.html?id=3ad483aafe5c4e3289bb48dd0d7c2f61 | Trails, paddling, rock climbing
| Recreation data | https://data-usfs.hub.arcgis.com/datasets/usfs::recreation-opportunities-feature-layer/about/ | Campsites, picnic areas 
| Race/ethnicity | https://www.arcgis.com/home/item.html?id=e3a7d2d3e5834b7eb6b1c2943141ced6 | Downloaded only California data
| Education | https://data2.nhgis.org/main | Table B15003
| Income | https://data2.nhgis.org/main | Table C17002

All files were downloaded and stored in the /data/ folder of the project directory.

# Setup

In [1]:
# Import libraries
import arcpy, os

In [2]:
# Set environment settings
arcpy.env.overwriteOutput = True

In [3]:
# Create file geodatabase to hold output
gdb_path = r"C:\Users\kathr\Documents\outdoor-alliance\san-gabriel\san-gabriel-analysis\sg_output.gdb"
if not arcpy.Exists(gdb_path):
    arcpy.management.CreateFileGDB(r"C:\Users\kathr\Documents\outdoor-alliance\san-gabriel\san-gabriel-analysis", "sg_output.gdb")
    
# Set workspace path
arcpy.env.workspace = gdb_path

# Read in Data

In [4]:
# Set path to folder where all data files are stored
data_folder = r"C:\Users\kathr\Documents\outdoor-alliance\san-gabriel\data"

## Convert data in CAD format to GDB

In [117]:
# Recreation and AOI data are in CAD format
# They have been downloaded from OA ArcOnline portal and converted to CAD files already. CAD files are stored in the /data/ folder

# https://pro.arcgis.com/en/pro-app/latest/tool-reference/conversion/cad-to-geodatabase.htm
cad_data = os.path.join(data_folder, "cad-datasets")

C:\Users\kathr\Documents\outdoor-alliance\san-gabriel\data\cad-datasets


In [111]:
reference_scale = 1000 # default value from geoprocessing tool

# For each CAD file in the directory, convert it to a geodatabase format
for path, dirs, files in os.walk(cad_data):
    for file in files:
        if file.endswith("dwg"): # CAD file
            input_cad_dataset = os.path.join(cad_data, file)
            # Set out_file_name to the name of the file with extension trimmed
            out_file_name = os.path.splitext(file)[0]
            # Call CADToGeodatabase and store in output gdb
            arcpy.conversion.CADToGeodatabase(input_cad_dataset, gdb_path, out_file_name, reference_scale)

## Roads

TIGER road data was manually added to the geodatabase by dragging and dropping the layers from the original ArcGIS Project (downloaded from ArcGIS Online using link in Prerequisites section) to the current project.

# Buffer around AOI

In [112]:
# Create a 90 mile buffer around the AOI
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/analysis/buffer.htm

sg_expansion = r"Prop_SG_Monument_Expansion\Proposed_San_Gabriel_National_Monument_Expansion"
sg_expansion_buffer = "sg_expansion_buffer_90mi"
buffer_dist = "90 Miles"
dissolve_option = "ALL"

In [ ]:
arcpy.analysis.Buffer(in_features = sg_expansion,
                      out_feature_class = sg_expansion_buffer,
                      buffer_distance_or_field = buffer_dist,
                      dissolve_option = dissolve_option)

# Clip & Process Data Layers

## Roads

TIGER roads are manually clipped due to the drag/drop way of importing. Each layer (primary, secondary, and local) was clipped to the 90 mile buffer around the AOI.

In [5]:
# Clip USFS Roads to buffer around AOI
usfs_roads = os.path.join(data_folder, "S_USA.RoadCore_FS.gdb\RoadCore_FS")
usfs_roads_clipped = "usfs_roads_clipped"

In [6]:
arcpy.analysis.Clip(usfs_roads, sg_expansion_buffer, usfs_roads_clipped)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\usfs_roads_clipped'>

In [48]:
# Merge all TIGER roads
# Because these layers come from the same data source, they have the same column names and don't need additional mapping
primary_roads_clipped = "primary_roads_200mi"
secondary_roads_clipped = "secondary_roads_200mi"
local_roads_clipped = "Local_Roads_90_mi"

In [61]:
# Create feature dataset to hold the new TIGER roads layer
sr = arcpy.Describe(primary_roads_clipped).spatialReference # get coordinate reference system
out_name = "sg_roads"
arcpy.management.CreateFeatureDataset(gdb_path, out_name, sr)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\sg_roads'>

In [50]:
# Create feature class to hold the merge of TIGER roads and USFS roads
all_roads_merge = os.path.join(gdb_path, r"sg_roads\all_roads_merge")

In [55]:
# Create field mapping object
field_mappings = arcpy.FieldMappings()

# Add input field for road name into new output field
map_name = arcpy.FieldMap()
map_name.addInputField(primary_roads_clipped, "Name")
map_name.addInputField(secondary_roads_clipped, "Name")
map_name.addInputField(local_roads_clipped, "Name")
map_name.addInputField(usfs_roads_clipped, "NAME")

# Set name of new output field for site name
road_name = map_name.outputField
road_name.name = "Road_Name"
map_name.outputField = road_name

# Add input field for road class into new output field
map_class = arcpy.FieldMap()
map_class.addInputField(primary_roads_clipped, "MTFCC")
map_class.addInputField(secondary_roads_clipped, "MTFCC")
map_class.addInputField(local_roads_clipped, "MTFCC")
map_class.addInputField(usfs_roads_clipped, "SYMBOL_CODE")


# Set name of new output field for site type
road_class = map_class.outputField
road_class.name = "Road_Class"
map_class.outputField = road_class

# Add output fields to field mappings object
field_mappings.addFieldMap(map_name)
field_mappings.addFieldMap(map_class)

In [56]:
# Merge TIGER and USFS roads together
arcpy.management.Merge([primary_roads_clipped, secondary_roads_clipped, local_roads_clipped, usfs_roads_clipped], 
                       all_roads_merge, 
                       field_mappings,
                      add_source = "ADD_SOURCE_INFO")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\all_roads_merge'>

## Visitors Centers

In [84]:
# Recreation Opportunities: clip to buffer around AOI
rec_ops = os.path.join(data_folder, "Recreation_Opportunities_(Feature_Layer)\Recreation_Opportunities_(Feature_Layer).shp")
rec_ops_clipped = "rec_ops_clipped"

In [8]:
arcpy.analysis.Clip(rec_ops, sg_expansion_buffer, rec_ops_clipped)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\rec_ops_clipped'>

In [23]:
# Recreation Sites: clip to buffer around AOI
rec_sites = os.path.join(data_folder, "Recreation_Sites_(Feature_Layer)\Recreation_Sites_(Feature_Layer).shp")
rec_sites_clipped = "rec_sites_clipped"

In [10]:
arcpy.analysis.Clip(rec_sites, sg_expansion_buffer, rec_sites_clipped)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\rec_sites_clipped'>

In [24]:
# Recreation Sites: filter to within 100 m of the AOI
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-location.htm
in_layer = rec_sites_clipped
overlap_type = "WITHIN_A_DISTANCE"
search_distance = "100 Meters"
selection_type = "NEW_SELECTION"
selecting_features = r"Prop_SG_Monument_Expansion\Proposed_San_Gabriel_National_Monument_Expansion"

aoi_rec_sites = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       search_distance = search_distance,
                                                       selection_type = selection_type)

In [25]:
# Recreation Sites: further select relevant site types
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-attribute.htm
in_layer = aoi_rec_sites
selection_type = "SUBSET_SELECTION"
where_clause = "SITE_SUBTY = 'INFO SITE/FEE STATION'"

arcpy.management.SelectLayerByAttribute(in_layer_or_view = in_layer, 
                                        selection_type = selection_type, 
                                        where_clause = where_clause)

<Result 'rec_sites_clipped_Layer2'>

In [26]:
# Recreation Opportunities: filter to within 100 m of the AOI
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-location.htm
in_layer = rec_ops_clipped
overlap_type = "WITHIN_A_DISTANCE"
search_distance = "100 Meters"
selection_type = "NEW_SELECTION"
selecting_features = r"Prop_SG_Monument_Expansion\Proposed_San_Gabriel_National_Monument_Expansion"

aoi_rec_ops = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       search_distance = search_distance,
                                                       selection_type = selection_type)

In [27]:
# Recreation Opportunities: further select relevant site types
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-attribute.htm
in_layer = aoi_rec_ops
selection_type = "SUBSET_SELECTION"
where_clause = "MARKERACTI = 'Visitor Centers' Or MARKERACTI = 'Visitor Programs'"

arcpy.management.SelectLayerByAttribute(in_layer_or_view = in_layer, 
                                        selection_type = selection_type, 
                                        where_clause = where_clause)

<Result 'rec_ops_clipped_Layer5'>

In [28]:
# Create new layers
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/copy-features.htm
sg_rec_sites = "sg_rec_sites"
sg_rec_ops = "sg_rec_ops"

arcpy.management.CopyFeatures(aoi_rec_sites, sg_rec_sites)
arcpy.management.CopyFeatures(aoi_rec_ops, sg_rec_ops)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\sg_rec_ops'>

In [29]:
# Merge all visitor centers together
visitor_centers = "visitor_centers"

# create field mapping object
field_mappings = arcpy.FieldMappings()

# Add input field for site name into new output field
map_site_name = arcpy.FieldMap()
map_site_name.addInputField(sg_rec_sites, "SITE_NAME")
map_site_name.addInputField(sg_rec_ops, "RECAREANAM")

# Set name of new output field for site name
site_name = map_site_name.outputField
site_name.name = "Site_Name"
map_site_name.outputField = site_name

# Add input field for site type into new output field
map_site_type = arcpy.FieldMap()
map_site_type.addInputField(sg_rec_sites, "SITE_SUBTY")
map_site_type.addInputField(sg_rec_ops, "MARKERACTI")

# Set name of new output field for site type
site_type = map_site_type.outputField
site_type.name = "Site_Type"
map_site_type.outputField = site_type

# add output fields to field mappings object
field_mappings.addFieldMap(map_site_name)
field_mappings.addFieldMap(map_site_type)

In [30]:
arcpy.management.Merge([sg_rec_ops, sg_rec_sites], visitor_centers, field_mappings)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\visitor_centers'>

## OA Recreation Data
* Climbing
* Whitewater paddling
* Trails

In [5]:
# Climbing: filter to within 100 m of the AOI
in_layer = r"Rock_Climbing\Rock_Climbing_Site"
overlap_type = "WITHIN_A_DISTANCE"
search_distance = "100 Meters"
selection_type = "NEW_SELECTION"
selecting_features = r"Prop_SG_Monument_Expansion\Proposed_San_Gabriel_National_Monument_Expansion"

aoi_climbing = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       search_distance = search_distance,
                                                       selection_type = selection_type)

# Create new layers
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/copy-features.htm
sg_climbing = "sg_climbing"

arcpy.management.CopyFeatures(aoi_climbing, sg_climbing)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\sg_climbing'>

In [6]:
# River Access: filter to within 100 m of the AOI
in_layer = r"River_Access\River_Access"
overlap_type = "WITHIN_A_DISTANCE"
search_distance = "100 Meters"
selection_type = "NEW_SELECTION"
selecting_features = r"Prop_SG_Monument_Expansion\Proposed_San_Gabriel_National_Monument_Expansion"

aoi_river = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       search_distance = search_distance,
                                                       selection_type = selection_type)

# Create new layers
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/copy-features.htm
sg_river_access = "sg_river_access"

arcpy.management.CopyFeatures(aoi_river, sg_river_access)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\sg_river_access'>

In [7]:
# Whitewater paddling runs: clip
aoi_ww = r"Whitewater_Paddling\Whitewater_Paddling"
sg_whitewater = "sg_whitewater"

arcpy.analysis.Clip(aoi_ww, r"Prop_SG_Monument_Expansion\Proposed_San_Gabriel_National_Monument_Expansion", sg_whitewater)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\sg_whitewater'>

In [71]:
# Recalculate length field after clipping
arcpy.management.CalculateGeometryAttributes(in_features = "sg_whitewater",
                                             geometry_property = [["Length_Miles", "LENGTH"]],
                                             length_unit = "MILES_INT")

<Result 'sg_whitewater'>

In [8]:
# Trails: clip
aoi_trails = r"Trails\Trails"
sg_trails = "sg_trails"

arcpy.analysis.Clip(aoi_trails, r"Prop_SG_Monument_Expansion\Proposed_San_Gabriel_National_Monument_Expansion", sg_trails)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\sg_trails'>

In [66]:
# Recalculate length field after clipping
arcpy.management.CalculateGeometryAttributes(in_features = "sg_trails",
                                             geometry_property = [["Length_Miles", "LENGTH"]],
                                             length_unit = "MILES_INT")

<Result 'sg_trails'>

## Picnic

In [174]:
# Recreation Opportunities: filter to within 100 m of the AOI
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-location.htm
in_layer = rec_ops_clipped
overlap_type = "WITHIN_A_DISTANCE"
search_distance = "100 Meters"
selection_type = "NEW_SELECTION"
selecting_features = r"Prop_SG_Monument_Expansion\Proposed_San_Gabriel_National_Monument_Expansion"

aoi_rec_ops = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       search_distance = search_distance,
                                                       selection_type = selection_type)

In [175]:
# Recreation Opportunities: further select relevant site types
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-attribute.htm
in_layer = aoi_rec_ops
selection_type = "SUBSET_SELECTION"
where_clause = "MARKERACTI = 'Picnicking'"

arcpy.management.SelectLayerByAttribute(in_layer_or_view = in_layer, 
                                        selection_type = selection_type, 
                                        where_clause = where_clause)

<Result 'rec_ops_clipped_Layer2'>

In [176]:
# Create new layers
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/copy-features.htm
sg_picnic = "sg_picnic"

arcpy.management.CopyFeatures(aoi_rec_ops, sg_picnic)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\sg_picnic'>

## Camping

In [177]:
# Recreation Opportunities: filter to within 100 m of the AOI
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-location.htm
in_layer = rec_ops_clipped
overlap_type = "WITHIN_A_DISTANCE"
search_distance = "100 Meters"
selection_type = "NEW_SELECTION"
selecting_features = r"Prop_SG_Monument_Expansion\Proposed_San_Gabriel_National_Monument_Expansion"

aoi_rec_ops = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       search_distance = search_distance,
                                                       selection_type = selection_type)

In [179]:
# Recreation Opportunities: further select relevant site types
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-attribute.htm
in_layer = aoi_rec_ops
selection_type = "SUBSET_SELECTION"
where_clause = "MARKERAC_1 = 'Camping & Cabins'"

arcpy.management.SelectLayerByAttribute(in_layer_or_view = in_layer, 
                                        selection_type = selection_type, 
                                        where_clause = where_clause)

<Result 'rec_ops_clipped_Layer5'>

In [180]:
# Create new layers
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/copy-features.htm
sg_camping = "sg_camping"

arcpy.management.CopyFeatures(aoi_rec_ops, sg_camping)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\sg_camping'>

## Census

### 2020 Redistricting Tracts (Racial Data)

In [72]:
# Clip census to buffer around AOI
census = os.path.join(data_folder, r"Census\Default.gdb\ca_census_2020")
census_clipped = "census_clipped"

In [45]:
arcpy.analysis.Clip(census, sg_expansion_buffer, census_clipped)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\census_clipped'>

In [74]:
# Import census for entire state of CA
ca_census = "ca_census"
arcpy.management.Copy(in_data = census,
                      out_data = ca_census,
                      data_type = "FeatureClass")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\ca_census'>